In [ ]:
import tkinter as tk
from tkinter import messagebox, filedialog
from googlesearch import search
import requests
from bs4 import BeautifulSoup
import re, csv, os
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from email import encoders


# Extract emails from text
def extract_emails(text):
    return list(set(re.findall(r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}", text)))

# Extract title or fallback to domain
def get_site_name(html, url):
    try:
        soup = BeautifulSoup(html, 'html.parser')
        return soup.title.string.strip()
    except:
        return url.split("//")[-1].split("/")[0]

# Scrape and collect data
def crawl_and_collect(query, max_results=10):
    results = search(query, num_results=max_results)
    final_data = []

    for url in results:
        try:
            res = requests.get(url, timeout=5)
            text = res.text
            soup = BeautifulSoup(text, 'html.parser')
            name = get_site_name(text, url)
            emails = extract_emails(soup.get_text())

            if emails:
                for email in emails:
                    final_data.append([name, email, url])
            else:
                final_data.append([name, "", url])
        except:
            continue
    return final_data

# Save to CSV
def save_to_csv(data, filename="results.csv"):
    os.makedirs("data", exist_ok=True)
    path = os.path.join("data", filename)
    with open(path, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["Name", "Email", "Source URL"])
        writer.writerows(data)
    return path

# Email the CSV
def send_email(sender, password, receiver, subject, body, filepath):
    msg = MIMEMultipart()
    msg["From"] = sender
    msg["To"] = receiver
    msg["Subject"] = subject
    msg.attach(MIMEText(body, "plain"))

    with open(filepath, "rb") as f:
        part = MIMEBase("application", "octet-stream")
        part.set_payload(f.read())
        encoders.encode_base64(part)
        part.add_header("Content-Disposition", f'attachment; filename="{os.path.basename(filepath)}"')
        msg.attach(part)

    with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
        server.login(sender, password)
        server.sendmail(sender, receiver, msg.as_string())

# ----- GUI Setup -----
def run_script():
    query = query_entry.get()
    if not query:
        messagebox.showwarning("Input Required", "Please enter a search query.")
        return

    status_label.config(text="Collecting data, please wait...")
    root.update()

    try:
        data = crawl_and_collect(query, max_results=10)
        if not data:
            status_label.config(text="No data found.")
            return

        file_path = save_to_csv(data)
        status_label.config(text=f"Data saved to: {file_path}")

        if send_email_var.get():
            sender = sender_entry.get()
            password = password_entry.get()
            receiver = receiver_entry.get()

            if not all([sender, password, receiver]):
                messagebox.showwarning("Missing Fields", "Fill all email fields to send.")
                return

            subject = f"Collected Leads for: {query}"
            body = f"""Hi,

Please find attached the leads collected for the query: "{query}"

This includes available contact emails and source URLs.

Best regards,
Hackveda Automation Bot
"""
            send_email(sender, password, receiver, subject, body, file_path)
            messagebox.showinfo("Email Sent", f"Email sent successfully to {receiver}")
    except Exception as e:
        status_label.config(text=f"Error: {e}")
        messagebox.showerror("Failed", str(e))


# Create GUI window
root = tk.Tk()
root.title("Smart Lead Collector")
root.geometry("500x450")

# Widgets
tk.Label(root, text="Enter search query:").pack(pady=5)
query_entry = tk.Entry(root, width=60)
query_entry.pack(pady=5)

send_email_var = tk.BooleanVar()
tk.Checkbutton(root, text="Send CSV to Email?", variable=send_email_var).pack(pady=5)

# Email Fields
tk.Label(root, text="Sender Gmail:").pack()
sender_entry = tk.Entry(root, width=40)
sender_entry.pack()

tk.Label(root, text="Gmail App Password:").pack()
password_entry = tk.Entry(root, show="*", width=40)
password_entry.pack()

tk.Label(root, text="Receiver Email:").pack()
receiver_entry = tk.Entry(root, width=40)
receiver_entry.pack()

tk.Button(root, text="Start Search", command=run_script, bg="#4CAF50", fg="white").pack(pady=20)

status_label = tk.Label(root, text="", fg="blue")
status_label.pack()

root.mainloop()
